---
# Preparation

In [10]:
# !jupyter nbconvert --output-dir="./final" --to script aicup_bert_lstm.ipynb

[NbConvertApp] Converting notebook aicup_bert_lstm.ipynb to script
[NbConvertApp] Writing 14892 bytes to aicup_bert_lstm.py


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


cuda:0


In [2]:
df = pd.read_csv('Batch_answers - train_data (no-blank).csv', encoding="ISO-8859-1")
df = df.drop(columns=['Unnamed: 6', 'total no.: 7987'])
# df_test = pd.read_csv('Batch_answers - test_data(no_label).csv', encoding="ISO-8859-1")
df

,id,q,r,s,q',r'
0,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True ."""
1,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""can go both ways . We all doubt . It is what ...","""True"""
2,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True"""
3,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""seem to support the killing of certain people""","""based on the idea that people are dispensible..."
4,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""you seem to support the killing of certain pe...","""based on the idea that people are dispensible"""
...,...,...,...,...,...,...
38341,10001,"""good thing this argument has never been done ...","""And teen sex does n't , by the very nature of...",DISAGREE,"""You are much better off making theft legal an...","""And teen sex does n't , by the very nature of..."
38342,10002,"""I know one thing , anything that happens , po...","""Was n't sinjin crowing about his plans to tak...",DISAGREE,"""I know one thing , anything that happens , po...","""Was n't sinjin crowing about his plans to tak..."
38343,10002,"""I know one thing , anything that happens , po...","""Was n't sinjin crowing about his plans to tak...",DISAGREE,"""FBI Arrests Three Men in Terror Plot that Tar...","""Was n't sinjin crowing about his plans to tak..."
38344,10003,"""I enjoy Botany more than most things and I ha...","""Hi Smallax , welcome to the forum . I did a s...",AGREE,"""I enjoy Botany more than most things and I ha...","""Hi Smallax , welcome to the forum . I did a s..."


---
# Data Processing

In [4]:
df[['q','r',"q'","r'"]] = df[['q','r',"q'","r'"]].apply(lambda x: x.str.strip('\"'))

# # augment 50%
df_aug = df.sample(n=int(0.5*len(df)), random_state=1)
df = pd.concat([df, df_aug], ignore_index=True)

    

df['r'] = df['s'] + ':' + df['r']
df['sub_q_true'] = [1 if x in y else 0 for x,y in zip(df["q'"],df["q"])]
df['sub_r_true'] = [1 if x in y else 0 for x,y in zip(df["r'"],df["r"])]
df['sub_both'] = df['sub_q_true']*df['sub_r_true']
df.head(3)

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1


In [5]:
data = df.loc[df['sub_both'] == 1]
data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
data['q_end'] = [x+len(y)-1 for x,y in zip(data["q_start"],data["q'"])]
data['r_end'] = [x+len(y)-1 for x,y in zip(data["r_start"],data["r'"])]
data

/tmp/ipykernel_860296/2710139891.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
/tmp/ipykernel_860296/2710139891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
/tmp/ipykernel_860296/2710139891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1,0,6,76,11
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1,3,6,74,9
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1,0,6,76,9
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1,17,6,61,98
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1,13,6,81,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57513,996,"If you want an answer to that , I suggest you ...","DISAGREE:What are you afraid of , Trebor ? ? T...",DISAGREE,"If you want an answer to that , I suggest you ...","What are you afraid of , Trebor ? ? The whole ...",1,1,1,0,9,59,111
57514,378,Now that 's a bald-faced lie,DISAGREE:No - it 's not . Your stand on rape v...,DISAGREE,Now that 's a bald-faced lie,No - it 's not . Your stand on rape victims pr...,1,1,1,0,9,27,61
57515,4002,You are making some false assumptions . Howeve...,"DISAGREE:Yep , he dodged it again . Care to ex...",DISAGREE,You are making some false assumptions .,"Yep , he dodged it again .",1,1,1,0,9,38,34
57517,5445,everything . it states that you can not discri...,DISAGREE:By that logic being black or of some ...,DISAGREE,thats discrimination,That 's not correct .,1,1,1,178,79,197,99


In [6]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(data, test_size=0.1)

---
# Tokenize

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [8]:
train_data_q = train['q'].tolist()
valid_data_q = valid['q'].tolist()

train_data_r = train['r'].tolist()
valid_data_r = valid['r'].tolist()

In [9]:
train_encodings = tokenizer(train_data_q, train_data_r, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_q, valid_data_r, truncation=True, padding=True)

In [10]:
train_answer = train[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')
valid_answer = valid[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')

In [11]:
def add_token_positions(encodings, answers):
    q_start, r_start, q_end, r_end = [],[],[],[]

    for i in range(len(answers)):
        # print(i)
        q_start.append(encodings.char_to_token(i, answers[i]['q_start'], 0))
        r_start.append(encodings.char_to_token(i, answers[i]['r_start'], 1))
        q_end.append(encodings.char_to_token(i, answers[i]['q_end'], 0))
        r_end.append(encodings.char_to_token(i, answers[i]['r_end'], 1))

        if q_start[-1] is None:
            q_start[-1] = 0
            q_end[-1] = 0
            # continue

        if r_start[-1] is None:
            r_start[-1] = 0
            r_end[-1] = 0
            # continue

        shift = 1
        while q_end[-1] is None:
            q_end[-1] = encodings.char_to_token(i, answers[i]['q_end'] - shift)
            shift += 1
        shift = 1
        while r_end[-1] is None:
            r_end[-1] = encodings.char_to_token(i, answers[i]['r_end'] - shift)
            shift += 1
    encodings.update({'q_start':q_start, 'r_start':r_start,	'q_end':q_end, 'r_end':r_end})

In [12]:
# Convert char_based_id to token_based_id
# Find the corresponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)
add_token_positions(val_encodings, valid_answer)

In [13]:
print(train_encodings.keys())
print(train_encodings['q_start'][0])
print(train_encodings['r_start'][0])
print(train_encodings['q_end'][0])
print(train_encodings['r_end'][0])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'q_start', 'r_start', 'q_end', 'r_end'])
3
54
24
78


---
# Dataset

In [14]:
class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [15]:
train_dataset = qrDataset(train_encodings)
val_dataset = qrDataset(val_encodings)

---
# Model

In [17]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.lstm = nn.LSTM(768, 256, batch_first=True, bidirectional=True, num_layers=2)
        self.fc = nn.Linear(256*2, 4)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        logits, _ = self.lstm(logits)
        # print(logits)
        # print(logits.shape)
        out = self.fc(logits)

        return out

In [20]:
from transformers import AdamW
from tqdm import tqdm
import torch
from transformers import AdamW, get_linear_schedule_with_warmup

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


epochs = 2
loss_fct = CrossEntropyLoss()
total_steps = len(train_loader) * epochs
model = myModel().to(device)
optim = AdamW(model.parameters(), lr=3e-5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/root/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

In [21]:
def evaluate(valid_loader):
    q_acc, r_acc = [], []
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            q_start = batch['q_start'].to(device)
            r_start = batch['r_start'].to(device)
            q_end = batch['q_end'].to(device)
            r_end = batch['r_end'].to(device)

            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 1)
            q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)
            
            
            q_start_pred = torch.argmax(q_start_logits, dim=1)
            r_start_pred = torch.argmax(r_start_logits, dim=1)
            q_end_pred = torch.argmax(q_end_logits, dim=1)
            r_end_pred = torch.argmax(r_end_logits, dim=1)

            q_start_logits = q_start_logits.squeeze(-1).contiguous()
            r_start_logits = r_start_logits.squeeze(-1).contiguous()
            q_end_logits = q_end_logits.squeeze(-1).contiguous()
            r_end_logits = r_end_logits.squeeze(-1).contiguous()

            q_start_loss = loss_fct(q_start_logits, q_start)
            r_start_loss = loss_fct(r_start_logits, r_start)
            q_end_loss = loss_fct(q_end_logits, q_end)
            r_end_loss = loss_fct(r_end_logits, r_end)

            loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss
            
            q_start = q_start.reshape([q_start.size(dim=0),1])
            r_start = r_start.reshape([r_start.size(dim=0),1])
            q_end = q_end.reshape([q_end.size(dim=0),1])
            r_end = r_end.reshape([r_end.size(dim=0),1])
            
            q_start_score = ((q_start_pred == q_start).sum()/len(q_start_pred)).item()
            r_start_score = ((r_start_pred == r_start).sum()/len(r_start_pred)).item() 
            q_end_score = ((q_end_pred == q_end).sum()/len(q_end_pred)).item() 
            r_end_score = ((r_end_pred == r_end).sum()/len(r_end_pred)).item() 
            
            q_acc.append(q_start_score)
            q_acc.append(q_end_score)
            r_acc.append(r_start_score)
            r_acc.append(r_end_score)

            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0
        q_acc = sum(q_acc)/len(q_acc)
        r_acc = sum(r_acc)/len(r_acc)

        print("evaluate loss: ", loss)
        print("q-acc: ", q_acc)
        print("r-acc: ", r_acc)

In [22]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        q_start = batch['q_start'].to(device)
        r_start = batch['r_start'].to(device)
        q_end = batch['q_end'].to(device)
        r_end = batch['r_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        # q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 1)
        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)
        
        

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_loss = loss_fct(q_start_logits, q_start)
        r_start_loss = loss_fct(r_start_logits, r_start)
        q_end_loss = loss_fct(q_end_logits, q_end)
        r_end_loss = loss_fct(r_end_logits, r_end)



        loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 50 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 50))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)

Epoch 0:   1% 51/5030 [00:20<32:42,  2.54it/s, loss=17.7]

Epoch 51 Batch 50 Loss 21.1067


Epoch 0:   2% 101/5030 [00:39<32:38,  2.52it/s, loss=10.6]

Epoch 101 Batch 100 Loss 13.2888


Epoch 0:   3% 151/5030 [00:59<32:29,  2.50it/s, loss=9.59]

...

Validation Epoch 511 Batch 510 Loss 6.6012


 97% 542/559 [01:15<00:02,  7.13it/s]

Validation Epoch 541 Batch 540 Loss 6.9131


100% 559/559 [01:18<00:00,  7.16it/s]

evaluate loss:  tensor(5.2688, device='cuda:0')
q-acc:  0.5036097623687738
r-acc:  0.49741247131061045


In [23]:
model_name = 'aicup_model_bert_base_bilstm'
torch.save(model.state_dict(), model_name)

---
# Inference

In [ ]:
# model_name = 'aicup_model_bert_base_bilstm'
# model.load_state_dict(torch.load(model_name))

In [1]:
df_test = pd.read_csv('Batch_answers - test_data(no_label).csv', encoding="ISO-8859-1")
df_test

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


,id,q,r,s
0,6199,"""-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ...","""If so , why do we still have apes , and why a...",DISAGREE
1,5807,"""There 's a lot of discussion there on that is...","""Of course . The makers of Expelled were withi...",DISAGREE
2,8487,"""`` It 's not helping . The guns these people ...","""Oh , I would wager about like Mexico , about ...",DISAGREE
3,1760,"""Shooting : 3 seriously injured in Auburn shoo...","""Pickup strikes group of four youths | Houston...",AGREE
4,6228,"""This is the argument concerning 'choice ' tha...","""I believe there is a point at which we ( soci...",DISAGREE
...,...,...,...,...
2011,9499,"""You are betraying your belief system .""","""Yep . ( I 'm assuming that by `` belief syste...",AGREE
2012,4611,"""You are in a loud minority , railing against ...","""Being in the minority or in the majority is i...",DISAGREE
2013,9328,"""You bet your XXX that 'd make me happy .""","""Well , first , I probably would n't bet my XX...",DISAGREE
2014,5225,"""you say `` f * * * the Constitution. ``""","""and gun nuts say f * * * the children when we...",DISAGREE


In [3]:
df_test[['q', 'r']] = df_test[['q', 'r']].apply(lambda x: x.str.strip('\"'))
df_test['r'] = df_test['s'] + ':' + df_test['r']
df_test

,id,q,r,s
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE
1,5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE
2,8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE
3,1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE
4,6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE
...,...,...,...,...
2011,9499,You are betraying your belief system .,AGREE:Yep . ( I 'm assuming that by `` belief ...,AGREE
2012,4611,"You are in a loud minority , railing against t...",DISAGREE:Being in the minority or in the major...,DISAGREE
2013,9328,You bet your XXX that 'd make me happy .,"DISAGREE:Well , first , I probably would n't b...",DISAGREE
2014,5225,you say `` f * * * the Constitution. ``,DISAGREE:and gun nuts say f * * * the children...,DISAGREE


---
## Tokenize

In [7]:
test_data_q = df_test['q'].tolist()
test_data_r = df_test['r'].tolist()

In [8]:
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)

---
## Dataset

In [10]:
test_dataset = qrDataset(test_encodings)

---
## DataLoader

In [12]:
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

---
## Predict

In [14]:
from tqdm import tqdm

def predict(test_loader):
    predict_pos = []

    model.eval()

    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_prdict = torch.argmax(q_start_logits, 1).cpu().numpy()
        r_start_prdict = torch.argmax(r_start_logits, 1).cpu().numpy()
        q_end_prdict = torch.argmax(q_end_logits, 1).cpu().numpy()
        r_end_prdict = torch.argmax(r_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((q_start_prdict[i].item(), r_start_prdict[i].item(), q_end_prdict[i].item(), r_end_prdict[i].item()))

            q_sub = tokenizer.decode(input_ids[i][q_start_prdict[i]:q_end_prdict[i]+1])
            r_sub = tokenizer.decode(input_ids[i][r_start_prdict[i]:r_end_prdict[i]+1])
            
            q_sub_output.append(q_sub)
            r_sub_output.append(r_sub)
    
    return q_sub_output, r_sub_output, predict_pos

In [15]:
q_sub_output, r_sub_output, predict_pos = predict(test_loader)

100% 252/252 [00:34<00:00,  7.28it/s]


In [16]:
def get_output_post_fn(test, q_sub_output, r_sub_output):
    q_sub, r_sub = [], []
    for i in range(len(test)):

        q_sub_pred = q_sub_output[i].split()
        r_sub_pred = r_sub_output[i].split()

        if q_sub_pred is None:
            q_sub_pred = []
        q_sub_error_index = q_sub_pred.index('[SEP]') if '[SEP]' in q_sub_pred else -1

        if q_sub_error_index != -1:
            q_sub_pred = q_sub_pred[:q_sub_error_index]

        temp = r_sub_pred.copy()
        if r_sub_pred is None:
            r_sub_pred = []
        else:
            for j in range(len(temp)):
                if temp[j] == '[SEP]':
                    r_sub_pred.remove('[SEP]')
                if temp[j] == '[PAD]':
                    r_sub_pred.remove('[PAD]')

        q_sub.append(' '.join(q_sub_pred))
        r_sub.append(' '.join(r_sub_pred))

    return q_sub, r_sub

In [17]:
q_sub, r_sub = get_output_post_fn(df_test, q_sub_output, r_sub_output)

In [18]:
df_test['q_sub'] = q_sub
df_test['r_sub'] = r_sub
df_test = df_test.set_index('id')
df_test

,q,r,s,q_sub,r_sub
id,,,,,
6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE,- - - - - - - - - - - - - - - - - - - - - - - ...,"If so, why do we still have apes, and why are ..."
5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE,There's a lot of discussion there on that issu...,The makers of Expelled were within their right...
8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE,"It's not helping. The guns these people have, ...","Oh, I would wager about like Mexico, about 80 ..."
1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE,Shooting : 3 seriously injured in Auburn shoot...,This is America! You don't need no stinkin'mot...
6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE,"This is the argument concerning'choice'that, t...",I believe there is a point at which we ( socie...
...,...,...,...,...,...
9499,You are betraying your belief system .,AGREE:Yep . ( I 'm assuming that by `` belief ...,AGREE,You are betraying your belief system.,Yep. ( I'm assuming that by ` ` belief system ...
4611,"You are in a loud minority , railing against t...",DISAGREE:Being in the minority or in the major...,DISAGREE,"You are in a loud minority, railing against th...",Being in the minority or in the majority is ir...
9328,You bet your XXX that 'd make me happy .,"DISAGREE:Well , first , I probably would n't b...",DISAGREE,You bet your XXX that'd make me happy.,"I probably wouldn't bet my XXX, but whatever y..."


---
# To CSV

In [19]:
df_result = pd.read_csv('submission template.csv')
df_result = df_result.set_index('id')
df_result = pd.merge(df_result, df_test[['q_sub', 'r_sub']], left_index=True, right_index=True)
df_result = df_result.reset_index()
df_result = df_result.drop(['q', 'r'], axis=1)
df_result = df_result.rename(columns={"q_sub": "q", "r_sub": "r"})
# df_result[["q", "r"]] = df_result[["q", "r"]].astype(str)
df_result.update('"' + df_result[['q', 'r']].astype(str) + '"')

df_result.to_csv('submission_bert_base_bilstm.csv', encoding="utf-8", sep=',', index=False)

In [20]:
df_result[:30]

,id,q,r
0,1,"""I got a good idea. however, they do tend to s...","""By your own admission you [UNK] â t'hung out'..."
1,2,"""Be sure to give your guns a big fat kiss toni...","""Actually, they didn't. The whole tragedy was ..."
2,3,"""One of the biggest arguments against gun cont...","""Not quite. To be more correct regarding gover..."
3,4,"""First of all, compare the'B'specimen in your ...","""Comparison I could've just circled the whole ..."
4,5,"""There are some incedents that are beyond your...","""Well yes."""
5,6,"""So the answer to your question is that there ...","""So which is it : the action is moral, the act..."
6,7,"""Legality does not matter. Religous implicatio...","""Exact, to the point, & amp ; beautiful."""
7,15,"""Correctly viewed, evolution is not a series o...",""""""
8,16,"""I was under the impression this article was r...","""You're not one of those people who doesn't co..."
9,17,"""And dont you get a kick out of SP calling her...","""Actually he doesn't believe in a liar God lik..."
